##### Copyright 2020 Google

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Simulate a large quantum circuit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/qsim/tutorials/q32d14"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/qsim/blob/master/docs/tutorials/q32d14.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/qsim/blob/master/docs/tutorials/q32d14.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/qsim/docs/tutorials/q32d14.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

Caution: Because of its size, this large circuit will not run on a default Google Colab kernel and is unlikely to run on your personal computer. Follow the [Quantum simulation on GCP](./qsimcirq_gcp.md) tutorial to run on a larger Google Cloud VM.

This tutorial builds a large circuit—with 32 qubits and a gate depth of 14—in the following steps:

1. Build: The circuit build has two steps:
  * Layout the grid of qubits
  * Define the gate operations on the grid of qubits
2. Run options: In the final cell, the _qsim_ simulator is run.
  * Set `options = {'t' = 16, 'v' = 3}` for 16 Cores,
  * and `Verbosity = 3`.
3. Run Calculate Amplitudes: The final step is running the circuit and calculating the amplitudes for two input bitstrings, as described by [qsim_amplitudes usage](./usage.md).


## Setup

Install the Cirq and qsimcirq packages:

In [2]:
try:
    import cirq
except ImportError:
    !pip install cirq --quiet
    import cirq

try:
    import qsimcirq
except ImportError:
    !pip install qsimcirq --quiet
    import qsimcirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.1/579.1 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.4/596.4 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

And import module dependencies for this tutorial:

In [3]:
import numpy as np
import time

## Build the circuit

This is a large circuit with 32 qubits, and a depth of 14 quantum gate operations. Each `cirq.GridQubit()` call creates a simulated qubit with a grid location defined by the argument. This results in the grid of qubits `q`. The circuit is made up of a sequence of gate operations on the qubit grid. This explains the size of the circuit definition: each of the 32 qubits has a sequence of 14 gate operations applied.

The output of `print(circuit)` results in what renders as a complete circuit diagram, with each qubit represented as a line and the gate operations represented as notations on these lines.

### Layout the grid of qubits

In [4]:
q = [
        cirq.GridQubit(1,5),
        cirq.GridQubit(1,6),
        cirq.GridQubit(1,7),
        cirq.GridQubit(2,4),
        cirq.GridQubit(2,5),
        cirq.GridQubit(2,6),
        cirq.GridQubit(2,7),
        cirq.GridQubit(3,3),
        cirq.GridQubit(3,4),
        cirq.GridQubit(3,5),
        cirq.GridQubit(3,6),
        cirq.GridQubit(3,7),
        cirq.GridQubit(4,2),
        cirq.GridQubit(4,3),
        cirq.GridQubit(4,4),
        cirq.GridQubit(4,5),
        cirq.GridQubit(4,6),
        cirq.GridQubit(4,7),
        cirq.GridQubit(5,1),
        cirq.GridQubit(5,2),
        cirq.GridQubit(5,3),
        cirq.GridQubit(5,4),
        cirq.GridQubit(5,5),
        cirq.GridQubit(5,6),
        cirq.GridQubit(6,1),
        cirq.GridQubit(6,2),
        cirq.GridQubit(6,3),
        cirq.GridQubit(6,4),
        cirq.GridQubit(6,5),
        cirq.GridQubit(7,2),
        cirq.GridQubit(7,3),
        cirq.GridQubit(7,4)
    ]

### Define the gate operations on the qubits

In [5]:
circuit = cirq.Circuit(
    [
    cirq.rz(rads=-2.079870303178702*np.pi).on(q[1]),
    cirq.rz(rads=2.0436918407499873*np.pi).on(q[2]),
    cirq.rz(rads=1.2371391697444234*np.pi).on(q[3]),
    cirq.rz(rads=-1.2825274365288457*np.pi).on(q[4]),
    cirq.rz(rads=-0.6529975013575373*np.pi).on(q[5]),
    cirq.rz(rads=0.21248377848559125*np.pi).on(q[6]),
    cirq.rz(rads=0.2767373377033284*np.pi).on(q[8]),
    cirq.rz(rads=-0.18492941569567625*np.pi).on(q[9]),
    cirq.rz(rads=0.02232591119805812*np.pi).on(q[10]),
    cirq.rz(rads=-0.030028573876142287*np.pi).on(q[11]),
    cirq.rz(rads=-0.8467509808142173*np.pi).on(q[12]),
    cirq.rz(rads=0.8164932597686655*np.pi).on(q[13]),
    cirq.rz(rads=-1.00125113388313*np.pi).on(q[14]),
    cirq.rz(rads=1.1224546746752684*np.pi).on(q[15]),
    cirq.rz(rads=-0.16310561378711827*np.pi).on(q[16]),
    cirq.rz(rads=0.1766183348870303*np.pi).on(q[17]),
    cirq.rz(rads=-0.22542387771877406*np.pi).on(q[19]),
    cirq.rz(rads=0.2814659583608806*np.pi).on(q[20]),
    cirq.rz(rads=-0.33113463396189063*np.pi).on(q[21]),
    cirq.rz(rads=0.40440704518468423*np.pi).on(q[22]),
    cirq.rz(rads=-0.4081262439699967*np.pi).on(q[25]),
    cirq.rz(rads=0.3666829187201306*np.pi).on(q[26]),
    cirq.rz(rads=-0.3507308388473503*np.pi).on(q[27]),
    cirq.rz(rads=0.37554649493270875*np.pi).on(q[28]),
    cirq.rz(rads=-1.4187954353764791*np.pi).on(q[29]),
    cirq.rz(rads=1.5102819373895253*np.pi).on(q[30]),
    cirq.rz(rads=1.3803105504474993*np.pi).on(q[1]),
    cirq.rz(rads=-1.4164890128762133*np.pi).on(q[2]),
    cirq.rz(rads=-0.7660705551087533*np.pi).on(q[3]),
    cirq.rz(rads=0.7206822883243308*np.pi).on(q[4]),
    cirq.rz(rads=1.3183560383893944*np.pi).on(q[5]),
    cirq.rz(rads=-1.7588697612613406*np.pi).on(q[6]),
    cirq.rz(rads=-0.6722145774944012*np.pi).on(q[8]),
    cirq.rz(rads=0.7640224995020534*np.pi).on(q[9]),
    cirq.rz(rads=0.5799079899133832*np.pi).on(q[10]),
    cirq.rz(rads=-0.5876106525914674*np.pi).on(q[11]),
    cirq.rz(rads=1.0843371101222938*np.pi).on(q[12]),
    cirq.rz(rads=-1.1145948311678457*np.pi).on(q[13]),
    cirq.rz(rads=0.7990757781248072*np.pi).on(q[14]),
    cirq.rz(rads=-0.6778722373326689*np.pi).on(q[15]),
    cirq.rz(rads=-1.6258237067659351*np.pi).on(q[16]),
    cirq.rz(rads=1.6393364278658469*np.pi).on(q[17]),
    cirq.rz(rads=0.7948295009385445*np.pi).on(q[19]),
    cirq.rz(rads=-0.7387874202964381*np.pi).on(q[20]),
    cirq.rz(rads=0.049341949396894985*np.pi).on(q[21]),
    cirq.rz(rads=0.02393046182589869*np.pi).on(q[22]),
    cirq.rz(rads=0.4710627118441926*np.pi).on(q[25]),
    cirq.rz(rads=-0.5125060370940587*np.pi).on(q[26]),
    cirq.rz(rads=2.1645856475342256*np.pi).on(q[27]),
    cirq.rz(rads=-2.1397699914488673*np.pi).on(q[28]),
    cirq.rz(rads=1.2773117920270392*np.pi).on(q[29]),
    cirq.rz(rads=-1.1858252900139932*np.pi).on(q[30]),
    cirq.rz(rads=-5.435868884042397*np.pi).on(q[0]),
    cirq.rz(rads=5.438497289344933*np.pi).on(q[1]),
    cirq.rz(rads=-5.19048555249959*np.pi).on(q[4]),
    cirq.rz(rads=5.170988862096221*np.pi).on(q[5]),
    cirq.rz(rads=2.5333591271878086*np.pi).on(q[7]),
    cirq.rz(rads=-2.4748096263683066*np.pi).on(q[8]),
    cirq.rz(rads=-4.480708067260001*np.pi).on(q[9]),
    cirq.rz(rads=4.525888267898699*np.pi).on(q[10]),
    cirq.rz(rads=2.135954522972214*np.pi).on(q[13]),
    cirq.rz(rads=-2.1822665205802965*np.pi).on(q[14]),
    cirq.rz(rads=-3.7780476633662574*np.pi).on(q[15]),
    cirq.rz(rads=3.817335880513747*np.pi).on(q[16]),
    cirq.rz(rads=-2.8819419896554686*np.pi).on(q[18]),
    cirq.rz(rads=2.9028256034569604*np.pi).on(q[19]),
    cirq.rz(rads=0.7811374803446167*np.pi).on(q[20]),
    cirq.rz(rads=-0.6780279413275597*np.pi).on(q[21]),
    cirq.rz(rads=1.863573798571082*np.pi).on(q[22]),
    cirq.rz(rads=-2.150412392135508*np.pi).on(q[23]),
    cirq.rz(rads=2.2532274955007456*np.pi).on(q[24]),
    cirq.rz(rads=-2.5360843333016145*np.pi).on(q[25]),
    cirq.rz(rads=2.3134893226730737*np.pi).on(q[26]),
    cirq.rz(rads=-2.238493420699622*np.pi).on(q[27]),
    cirq.rz(rads=1.42630741834175*np.pi).on(q[30]),
    cirq.rz(rads=-1.5270341780432073*np.pi).on(q[31]),
    cirq.rz(rads=5.79385605258612*np.pi).on(q[0]),
    cirq.rz(rads=-5.791227647283584*np.pi).on(q[1]),
    cirq.rz(rads=5.223139057027918*np.pi).on(q[4]),
    cirq.rz(rads=-5.242635747431287*np.pi).on(q[5]),
    cirq.rz(rads=-2.346072351850546*np.pi).on(q[7]),
    cirq.rz(rads=2.404621852670048*np.pi).on(q[8]),
    cirq.rz(rads=5.048199817882042*np.pi).on(q[9]),
    cirq.rz(rads=-5.0030196172433445*np.pi).on(q[10]),
    cirq.rz(rads=-2.6543362735839113*np.pi).on(q[13]),
    cirq.rz(rads=2.6080242759758283*np.pi).on(q[14]),
    cirq.rz(rads=3.9045088495271663*np.pi).on(q[15]),
    cirq.rz(rads=-3.8652206323796765*np.pi).on(q[16]),
    cirq.rz(rads=1.9770644223044243*np.pi).on(q[18]),
    cirq.rz(rads=-1.9561808085029322*np.pi).on(q[19]),
    cirq.rz(rads=-1.5516585295358842*np.pi).on(q[20]),
    cirq.rz(rads=1.6547680685529413*np.pi).on(q[21]),
    cirq.rz(rads=-1.8933072151541963*np.pi).on(q[22]),
    cirq.rz(rads=1.6064686215897703*np.pi).on(q[23]),
    cirq.rz(rads=-0.5449135022758093*np.pi).on(q[24]),
    cirq.rz(rads=0.2620566644749405*np.pi).on(q[25]),
    cirq.rz(rads=-2.3490397609251703*np.pi).on(q[26]),
    cirq.rz(rads=2.424035662898622*np.pi).on(q[27]),
    cirq.rz(rads=-1.8655832225378013*np.pi).on(q[30]),
    cirq.rz(rads=1.7648564628363437*np.pi).on(q[31]),
    cirq.rz(rads=-6.214223110662173*np.pi).on(q[3]),
    cirq.rz(rads=6.24431588336413*np.pi).on(q[8]),
    cirq.rz(rads=-6.196295096608877*np.pi).on(q[4]),
    cirq.rz(rads=6.191833422443152*np.pi).on(q[9]),
    cirq.rz(rads=-5.367868774756692*np.pi).on(q[5]),
    cirq.rz(rads=5.257156584109544*np.pi).on(q[10]),
    cirq.rz(rads=-1.6118072404137829*np.pi).on(q[6]),
    cirq.rz(rads=1.5665192386902935*np.pi).on(q[11]),
    cirq.rz(rads=-5.408932498710608*np.pi).on(q[12]),
    cirq.rz(rads=5.396221422935972*np.pi).on(q[19]),
    cirq.rz(rads=-3.2786928385561493*np.pi).on(q[13]),
    cirq.rz(rads=3.339006443218924*np.pi).on(q[20]),
    cirq.rz(rads=-5.390755870544794*np.pi).on(q[14]),
    cirq.rz(rads=5.4172568990486605*np.pi).on(q[21]),
    cirq.rz(rads=-5.620144773112766*np.pi).on(q[15]),
    cirq.rz(rads=5.630469153514815*np.pi).on(q[22]),
    cirq.rz(rads=4.367652291347506*np.pi).on(q[16]),
    cirq.rz(rads=-3.9105776028384707*np.pi).on(q[23]),
    cirq.rz(rads=7.0181466269225865*np.pi).on(q[25]),
    cirq.rz(rads=-7.000766026200176*np.pi).on(q[29]),
    cirq.rz(rads=5.700873278515409*np.pi).on(q[26]),
    cirq.rz(rads=-5.683378195921049*np.pi).on(q[30]),
    cirq.rz(rads=4.586335789661189*np.pi).on(q[27]),
    cirq.rz(rads=-4.76537552715921*np.pi).on(q[31]),
    cirq.rz(rads=6.89944406229822*np.pi).on(q[3]),
    cirq.rz(rads=-6.869351289596263*np.pi).on(q[8]),
    cirq.rz(rads=6.506615138479995*np.pi).on(q[4]),
    cirq.rz(rads=-6.511076812645719*np.pi).on(q[9]),
    cirq.rz(rads=6.150506057270183*np.pi).on(q[5]),
    cirq.rz(rads=-6.2612182479173315*np.pi).on(q[10]),
    cirq.rz(rads=2.4087294851133443*np.pi).on(q[6]),
    cirq.rz(rads=-2.4540174868368334*np.pi).on(q[11]),
    cirq.rz(rads=4.737705877923889*np.pi).on(q[12]),
    cirq.rz(rads=-4.750416953698525*np.pi).on(q[19]),
    cirq.rz(rads=2.9425087256630427*np.pi).on(q[13]),
    cirq.rz(rads=-2.882195121000268*np.pi).on(q[20]),
    cirq.rz(rads=4.466531408750767*np.pi).on(q[14]),
    cirq.rz(rads=-4.440030380246901*np.pi).on(q[21]),
    cirq.rz(rads=4.486471496440378*np.pi).on(q[15]),
    cirq.rz(rads=-4.476147116038329*np.pi).on(q[22]),
    cirq.rz(rads=-4.89701654221443*np.pi).on(q[16]),
    cirq.rz(rads=5.354091230723465*np.pi).on(q[23]),
    cirq.rz(rads=-5.629287261948809*np.pi).on(q[25]),
    cirq.rz(rads=5.646667862671219*np.pi).on(q[29]),
    cirq.rz(rads=-5.760627714067928*np.pi).on(q[26]),
    cirq.rz(rads=5.778122796662288*np.pi).on(q[30]),
    cirq.rz(rads=-3.985782702743221*np.pi).on(q[27]),
    cirq.rz(rads=3.806742965245199*np.pi).on(q[31]),
    cirq.rz(rads=-2.4865845873665364*np.pi).on(q[0]),
    cirq.rz(rads=2.4890814068883764*np.pi).on(q[4]),
    cirq.rz(rads=-2.4240781150731663*np.pi).on(q[1]),
    cirq.rz(rads=2.419398026235366*np.pi).on(q[5]),
    cirq.rz(rads=2.3861256785493166*np.pi).on(q[2]),
    cirq.rz(rads=-2.392456163642626*np.pi).on(q[6]),
    cirq.rz(rads=12.703597923836748*np.pi).on(q[7]),
    cirq.rz(rads=-12.7869629079138*np.pi).on(q[13]),
    cirq.rz(rads=12.184253063938954*np.pi).on(q[8]),
    cirq.rz(rads=-12.108584830758572*np.pi).on(q[14]),
    cirq.rz(rads=3.782562501914174*np.pi).on(q[9]),
    cirq.rz(rads=-3.873596611893716*np.pi).on(q[15]),
    cirq.rz(rads=4.772639843256901*np.pi).on(q[10]),
    cirq.rz(rads=-4.771314675186062*np.pi).on(q[16]),
    cirq.rz(rads=8.49593730829863*np.pi).on(q[11]),
    cirq.rz(rads=-8.479908941862229*np.pi).on(q[17]),
    cirq.rz(rads=1.639481743922408*np.pi).on(q[18]),
    cirq.rz(rads=-1.9319083897827265*np.pi).on(q[24]),
    cirq.rz(rads=9.60223181672896*np.pi).on(q[19]),
    cirq.rz(rads=-9.605639326034064*np.pi).on(q[25]),
    cirq.rz(rads=6.330499004273446*np.pi).on(q[20]),
    cirq.rz(rads=-6.2177071019033425*np.pi).on(q[26]),
    cirq.rz(rads=9.851852381617888*np.pi).on(q[21]),
    cirq.rz(rads=-9.926465199012979*np.pi).on(q[27]),
    cirq.rz(rads=6.431104618355057*np.pi).on(q[22]),
    cirq.rz(rads=-6.38660616379351*np.pi).on(q[28]),
    cirq.rz(rads=2.557874433792943*np.pi).on(q[0]),
    cirq.rz(rads=-2.555377614271102*np.pi).on(q[4]),
    cirq.rz(rads=1.9789952328325573*np.pi).on(q[1]),
    cirq.rz(rads=-1.9836753216703575*np.pi).on(q[5]),
    cirq.rz(rads=-2.805807436079691*np.pi).on(q[2]),
    cirq.rz(rads=2.7994769509863815*np.pi).on(q[6]),
    cirq.rz(rads=-12.477250219528523*np.pi).on(q[7]),
    cirq.rz(rads=12.39388523545147*np.pi).on(q[13]),
    cirq.rz(rads=-11.31088974563283*np.pi).on(q[8]),
    cirq.rz(rads=11.386557978813212*np.pi).on(q[14]),
    cirq.rz(rads=-5.4898636407973544*np.pi).on(q[9]),
    cirq.rz(rads=5.398829530817813*np.pi).on(q[15]),
    cirq.rz(rads=-5.863871460773714*np.pi).on(q[10]),
    cirq.rz(rads=5.8651966288445525*np.pi).on(q[16]),
    cirq.rz(rads=-8.850693052252502*np.pi).on(q[11]),
    cirq.rz(rads=8.866721418688904*np.pi).on(q[17]),
    cirq.rz(rads=-2.40381552479658*np.pi).on(q[18]),
    cirq.rz(rads=2.1113888789362614*np.pi).on(q[24]),
    cirq.rz(rads=-10.03456101076628*np.pi).on(q[19]),
    cirq.rz(rads=10.031153501461176*np.pi).on(q[25]),
    cirq.rz(rads=-5.434421382024706*np.pi).on(q[20]),
    cirq.rz(rads=5.54721328439481*np.pi).on(q[26]),
    cirq.rz(rads=-9.17988634353845*np.pi).on(q[21]),
    cirq.rz(rads=9.10527352614336*np.pi).on(q[27]),
    cirq.rz(rads=-6.5670035038476025*np.pi).on(q[22]),
    cirq.rz(rads=6.61150195840915*np.pi).on(q[28]),
    cirq.rz(rads=-13.031870303178678*np.pi).on(q[1]),
    cirq.rz(rads=12.995691840749963*np.pi).on(q[2]),
    cirq.rz(rads=5.381139169744492*np.pi).on(q[3]),
    cirq.rz(rads=-5.426527436528915*np.pi).on(q[4]),
    cirq.rz(rads=-6.86899750135751*np.pi).on(q[5]),
    cirq.rz(rads=6.428483778485565*np.pi).on(q[6]),
    cirq.rz(rads=5.16073733770325*np.pi).on(q[8]),
    cirq.rz(rads=-5.068929415695599*np.pi).on(q[9]),
    cirq.rz(rads=-0.7176740888019262*np.pi).on(q[10]),
    cirq.rz(rads=0.7099714261238419*np.pi).on(q[11]),
    cirq.rz(rads=-4.694750980814187*np.pi).on(q[12]),
    cirq.rz(rads=4.664493259768636*np.pi).on(q[13]),
    cirq.rz(rads=-4.701251133883051*np.pi).on(q[14]),
    cirq.rz(rads=4.82245467467519*np.pi).on(q[15]),
    cirq.rz(rads=3.5368943862129347*np.pi).on(q[16]),
    cirq.rz(rads=-3.523381665113022*np.pi).on(q[17]),
    cirq.rz(rads=-1.113423877718808*np.pi).on(q[19]),
    cirq.rz(rads=1.1694659583609144*np.pi).on(q[20]),
    cirq.rz(rads=-3.587134633961795*np.pi).on(q[21]),
    cirq.rz(rads=3.6604070451845887*np.pi).on(q[22]),
    cirq.rz(rads=-5.2921262439699195*np.pi).on(q[25]),
    cirq.rz(rads=5.250682918720053*np.pi).on(q[26]),
    cirq.rz(rads=-6.349327548997941*np.pi).on(q[27]),
    cirq.rz(rads=6.3741432050833*np.pi).on(q[28]),
    cirq.rz(rads=-7.486795435376533*np.pi).on(q[29]),
    cirq.rz(rads=7.578281937389579*np.pi).on(q[30]),
    cirq.rz(rads=12.332310550447476*np.pi).on(q[1]),
    cirq.rz(rads=-12.36848901287619*np.pi).on(q[2]),
    cirq.rz(rads=-4.910070555108823*np.pi).on(q[3]),
    cirq.rz(rads=4.864682288324399*np.pi).on(q[4]),
    cirq.rz(rads=7.534356038389369*np.pi).on(q[5]),
    cirq.rz(rads=-7.974869761261314*np.pi).on(q[6]),
    cirq.rz(rads=-5.556214577494324*np.pi).on(q[8]),
    cirq.rz(rads=5.648022499501975*np.pi).on(q[9]),
    cirq.rz(rads=1.3199079899133674*np.pi).on(q[10]),
    cirq.rz(rads=-1.3276106525914517*np.pi).on(q[11]),
    cirq.rz(rads=4.932337110122265*np.pi).on(q[12]),
    cirq.rz(rads=-4.9625948311678165*np.pi).on(q[13]),
    cirq.rz(rads=4.499075778124728*np.pi).on(q[14]),
    cirq.rz(rads=-4.37787223733259*np.pi).on(q[15]),
    cirq.rz(rads=-5.325823706765988*np.pi).on(q[16]),
    cirq.rz(rads=5.3393364278658995*np.pi).on(q[17]),
    cirq.rz(rads=1.682829500938578*np.pi).on(q[19]),
    cirq.rz(rads=-1.6267874202964716*np.pi).on(q[20]),
    cirq.rz(rads=3.305341949396799*np.pi).on(q[21]),
    cirq.rz(rads=-3.232069538174005*np.pi).on(q[22]),
    cirq.rz(rads=5.3550627118441145*np.pi).on(q[25]),
    cirq.rz(rads=-5.39650603709398*np.pi).on(q[26]),
    cirq.rz(rads=8.163182357684818*np.pi).on(q[27]),
    cirq.rz(rads=-8.138366701599459*np.pi).on(q[28]),
    cirq.rz(rads=7.345311792027093*np.pi).on(q[29]),
    cirq.rz(rads=-7.253825290014047*np.pi).on(q[30]),
    cirq.rz(rads=-17.867868884042345*np.pi).on(q[0]),
    cirq.rz(rads=17.87049728934488*np.pi).on(q[1]),
    cirq.rz(rads=-17.622485552499665*np.pi).on(q[4]),
    cirq.rz(rads=17.602988862096296*np.pi).on(q[5]),
    cirq.rz(rads=7.565359127187911*np.pi).on(q[7]),
    cirq.rz(rads=-7.506809626368408*np.pi).on(q[8]),
    cirq.rz(rads=-15.28470806725993*np.pi).on(q[9]),
    cirq.rz(rads=15.329888267898626*np.pi).on(q[10]),
    cirq.rz(rads=7.019954522972137*np.pi).on(q[13]),
    cirq.rz(rads=-7.066266520580219*np.pi).on(q[14]),
    cirq.rz(rads=-13.842047663366333*np.pi).on(q[15]),
    cirq.rz(rads=13.881335880513822*np.pi).on(q[16]),
    cirq.rz(rads=-7.765941989655391*np.pi).on(q[18]),
    cirq.rz(rads=7.786825603456883*np.pi).on(q[19]),
    cirq.rz(rads=3.001137480344569*np.pi).on(q[20]),
    cirq.rz(rads=-2.8980279413275123*np.pi).on(q[21]),
    cirq.rz(rads=5.563573798571002*np.pi).on(q[22]),
    cirq.rz(rads=-5.8504123921354285*np.pi).on(q[23]),
    cirq.rz(rads=5.509227495500649*np.pi).on(q[24]),
    cirq.rz(rads=-5.792084333301517*np.pi).on(q[25]),
    cirq.rz(rads=7.868086032823645*np.pi).on(q[26]),
    cirq.rz(rads=-7.793090130850194*np.pi).on(q[27]),
    cirq.rz(rads=4.3863074183418185*np.pi).on(q[30]),
    cirq.rz(rads=-4.487034178043276*np.pi).on(q[31]),
    cirq.rz(rads=18.225856052586064*np.pi).on(q[0]),
    cirq.rz(rads=-18.223227647283533*np.pi).on(q[1]),
    cirq.rz(rads=17.655139057028*np.pi).on(q[4]),
    cirq.rz(rads=-17.674635747431363*np.pi).on(q[5]),
    cirq.rz(rads=-7.378072351850649*np.pi).on(q[7]),
    cirq.rz(rads=7.436621852670151*np.pi).on(q[8]),
    cirq.rz(rads=15.852199817881967*np.pi).on(q[9]),
    cirq.rz(rads=-15.80701961724327*np.pi).on(q[10]),
    cirq.rz(rads=-7.538336273583833*np.pi).on(q[13]),
    cirq.rz(rads=7.492024275975751*np.pi).on(q[14]),
    cirq.rz(rads=13.968508849527241*np.pi).on(q[15]),
    cirq.rz(rads=-13.929220632379753*np.pi).on(q[16]),
    cirq.rz(rads=6.861064422304347*np.pi).on(q[18]),
    cirq.rz(rads=-6.840180808502855*np.pi).on(q[19]),
    cirq.rz(rads=-3.771658529535837*np.pi).on(q[20]),
    cirq.rz(rads=3.874768068552894*np.pi).on(q[21]),
    cirq.rz(rads=-5.593307215154117*np.pi).on(q[22]),
    cirq.rz(rads=5.30646862158969*np.pi).on(q[23]),
    cirq.rz(rads=-3.800913502275713*np.pi).on(q[24]),
    cirq.rz(rads=3.5180566644748446*np.pi).on(q[25]),
    cirq.rz(rads=-7.9036364710757425*np.pi).on(q[26]),
    cirq.rz(rads=7.978632373049194*np.pi).on(q[27]),
    cirq.rz(rads=-4.825583222537869*np.pi).on(q[30]),
    cirq.rz(rads=4.724856462836412*np.pi).on(q[31]),
    cirq.rz(rads=-16.574223110662086*np.pi).on(q[3]),
    cirq.rz(rads=16.60431588336404*np.pi).on(q[8]),
    cirq.rz(rads=-15.816295096608934*np.pi).on(q[4]),
    cirq.rz(rads=15.811833422443211*np.pi).on(q[9]),
    cirq.rz(rads=-13.3598687747566*np.pi).on(q[5]),
    cirq.rz(rads=13.249156584109453*np.pi).on(q[10]),
    cirq.rz(rads=-4.127807240413703*np.pi).on(q[6]),
    cirq.rz(rads=4.082519238690215*np.pi).on(q[11]),
    cirq.rz(rads=-13.252932498710596*np.pi).on(q[12]),
    cirq.rz(rads=13.24022142293596*np.pi).on(q[19]),
    cirq.rz(rads=-8.162692838556204*np.pi).on(q[13]),
    cirq.rz(rads=8.223006443218978*np.pi).on(q[20]),
    cirq.rz(rads=-12.938755870544817*np.pi).on(q[14]),
    cirq.rz(rads=12.965256899048683*np.pi).on(q[21]),
    cirq.rz(rads=-12.724144773112773*np.pi).on(q[15]),
    cirq.rz(rads=12.73446915351482*np.pi).on(q[22]),
    cirq.rz(rads=11.027652291347495*np.pi).on(q[16]),
    cirq.rz(rads=-10.570577602838458*np.pi).on(q[23]),
    cirq.rz(rads=17.082146626922658*np.pi).on(q[25]),
    cirq.rz(rads=-17.06476602620025*np.pi).on(q[29]),
    cirq.rz(rads=14.58087327851535*np.pi).on(q[26]),
    cirq.rz(rads=-14.563378195920992*np.pi).on(q[30]),
    cirq.rz(rads=10.871739079510629*np.pi).on(q[27]),
    cirq.rz(rads=-11.050778817008649*np.pi).on(q[31]),
    cirq.rz(rads=-17.229351289596174*np.pi).on(q[8]),
    cirq.rz(rads=16.126615138480055*np.pi).on(q[4]),
    cirq.rz(rads=-16.131076812645777*np.pi).on(q[9]),
    cirq.rz(rads=14.142506057270092*np.pi).on(q[5]),
    cirq.rz(rads=-14.253218247917241*np.pi).on(q[10]),
    cirq.rz(rads=4.924729485113265*np.pi).on(q[6]),
    cirq.rz(rads=-4.9700174868367535*np.pi).on(q[11]),
    cirq.rz(rads=12.581705877923879*np.pi).on(q[12]),
    cirq.rz(rads=-12.594416953698515*np.pi).on(q[19]),
    cirq.rz(rads=7.826508725663096*np.pi).on(q[13]),
    cirq.rz(rads=-7.7661951210003215*np.pi).on(q[20]),
    cirq.rz(rads=12.014531408750791*np.pi).on(q[14]),
    cirq.rz(rads=-11.988030380246926*np.pi).on(q[21]),
    cirq.rz(rads=11.590471496440383*np.pi).on(q[15]),
    cirq.rz(rads=-11.580147116038336*np.pi).on(q[22]),
    cirq.rz(rads=-11.55701654221442*np.pi).on(q[16]),
    cirq.rz(rads=12.014091230723457*np.pi).on(q[23]),
    cirq.rz(rads=-15.693287261948884*np.pi).on(q[25]),
    cirq.rz(rads=15.710667862671292*np.pi).on(q[29]),
    cirq.rz(rads=-14.640627714067872*np.pi).on(q[26]),
    cirq.rz(rads=14.658122796662232*np.pi).on(q[30]),
    cirq.rz(rads=-10.271185992592658*np.pi).on(q[27]),
    cirq.rz(rads=10.092146255094638*np.pi).on(q[31]),
    cirq.rz(rads=-4.706584587366488*np.pi).on(q[0]),
    cirq.rz(rads=4.709081406888329*np.pi).on(q[4]),
    cirq.rz(rads=-4.644078115073251*np.pi).on(q[1]),
    cirq.rz(rads=4.639398026235451*np.pi).on(q[5]),
    cirq.rz(rads=4.902125678549236*np.pi).on(q[2]),
    cirq.rz(rads=-4.908456163642546*np.pi).on(q[6]),
    cirq.rz(rads=26.023597923836856*np.pi).on(q[7]),
    cirq.rz(rads=-26.106962907913907*np.pi).on(q[13]),
    cirq.rz(rads=25.356253063938887*np.pi).on(q[8]),
    cirq.rz(rads=-25.2805848307585*np.pi).on(q[14]),
    cirq.rz(rads=8.370562501914259*np.pi).on(q[9]),
    cirq.rz(rads=-8.461596611893802*np.pi).on(q[15]),
    cirq.rz(rads=10.100639843256841*np.pi).on(q[10]),
    cirq.rz(rads=-10.099314675186001*np.pi).on(q[16]),
    cirq.rz(rads=18.263937308298605*np.pi).on(q[11]),
    cirq.rz(rads=-18.247908941862203*np.pi).on(q[17]),
    cirq.rz(rads=4.303481743922509*np.pi).on(q[18]),
    cirq.rz(rads=-4.595908389782827*np.pi).on(q[24]),
    cirq.rz(rads=20.40623181672889*np.pi).on(q[19]),
    cirq.rz(rads=-20.409639326033993*np.pi).on(q[25]),
    cirq.rz(rads=13.138499004273484*np.pi).on(q[20]),
    cirq.rz(rads=-13.02570710190338*np.pi).on(q[26]),
    cirq.rz(rads=19.994449091768548*np.pi).on(q[21]),
    cirq.rz(rads=-20.069061909163636*np.pi).on(q[27]),
    cirq.rz(rads=13.831104618355031*np.pi).on(q[22]),
    cirq.rz(rads=-13.786606163793484*np.pi).on(q[28]),
    cirq.rz(rads=4.777874433792896*np.pi).on(q[0]),
    cirq.rz(rads=-4.775377614271054*np.pi).on(q[4]),
    cirq.rz(rads=4.198995232832642*np.pi).on(q[1]),
    cirq.rz(rads=-4.203675321670441*np.pi).on(q[5]),
    cirq.rz(rads=-5.321807436079611*np.pi).on(q[2]),
    cirq.rz(rads=5.315476950986302*np.pi).on(q[6]),
    cirq.rz(rads=-25.79725021952863*np.pi).on(q[7]),
    cirq.rz(rads=25.713885235451578*np.pi).on(q[13]),
    cirq.rz(rads=-24.48288974563276*np.pi).on(q[8]),
    cirq.rz(rads=24.55855797881315*np.pi).on(q[14]),
    cirq.rz(rads=-10.07786364079744*np.pi).on(q[9]),
    cirq.rz(rads=9.986829530817898*np.pi).on(q[15]),
    cirq.rz(rads=-11.191871460773655*np.pi).on(q[10]),
    cirq.rz(rads=11.193196628844492*np.pi).on(q[16]),
    cirq.rz(rads=-18.61869305225248*np.pi).on(q[11]),
    cirq.rz(rads=18.63472141868888*np.pi).on(q[17]),
    cirq.rz(rads=-5.067815524796681*np.pi).on(q[18]),
    cirq.rz(rads=4.775388878936363*np.pi).on(q[24]),
    cirq.rz(rads=-20.83856101076621*np.pi).on(q[19]),
    cirq.rz(rads=20.835153501461107*np.pi).on(q[25]),
    cirq.rz(rads=-12.242421382024746*np.pi).on(q[20]),
    cirq.rz(rads=12.35521328439485*np.pi).on(q[26]),
    cirq.rz(rads=-19.32248305368911*np.pi).on(q[21]),
    cirq.rz(rads=19.24787023629402*np.pi).on(q[27]),
    cirq.rz(rads=-13.967003503847575*np.pi).on(q[22]),
    cirq.rz(rads=14.01150195840912*np.pi).on(q[28]),
    cirq.rz(rads=-23.983870303178655*np.pi).on(q[1]),
    cirq.rz(rads=23.947691840749943*np.pi).on(q[2]),
    cirq.rz(rads=9.52513916974456*np.pi).on(q[3]),
    cirq.rz(rads=-9.570527436528984*np.pi).on(q[4]),
    cirq.rz(rads=-13.084997501357485*np.pi).on(q[5]),
    cirq.rz(rads=12.644483778485537*np.pi).on(q[6]),
    cirq.rz(rads=10.044737337703173*np.pi).on(q[8]),
    cirq.rz(rads=-9.952929415695523*np.pi).on(q[9]),
    cirq.rz(rads=-1.4576740888019104*np.pi).on(q[10]),
    cirq.rz(rads=1.4499714261238263*np.pi).on(q[11]),
    cirq.rz(rads=-8.542750980814159*np.pi).on(q[12]),
    cirq.rz(rads=8.512493259768608*np.pi).on(q[13]),
    cirq.rz(rads=-8.401251133882973*np.pi).on(q[14]),
    cirq.rz(rads=8.52245467467511*np.pi).on(q[15]),
    cirq.rz(rads=7.236894386212986*np.pi).on(q[16]),
    cirq.rz(rads=-7.223381665113074*np.pi).on(q[17]),
    cirq.rz(rads=-2.0014238777188416*np.pi).on(q[19]),
    cirq.rz(rads=2.057465958360948*np.pi).on(q[20]),
    cirq.rz(rads=-6.843134633961698*np.pi).on(q[21]),
    cirq.rz(rads=6.916407045184491*np.pi).on(q[22]),
    cirq.rz(rads=-10.176126243969842*np.pi).on(q[25]),
    cirq.rz(rads=10.134682918719976*np.pi).on(q[26]),
    cirq.rz(rads=-12.347924259148533*np.pi).on(q[27]),
    cirq.rz(rads=12.372739915233888*np.pi).on(q[28]),
    cirq.rz(rads=-13.554795435376587*np.pi).on(q[29]),
    cirq.rz(rads=13.646281937389634*np.pi).on(q[30]),
    cirq.rz(rads=23.28431055044745*np.pi).on(q[1]),
    cirq.rz(rads=-23.320489012876163*np.pi).on(q[2]),
    cirq.rz(rads=-9.054070555108892*np.pi).on(q[3]),
    cirq.rz(rads=9.008682288324469*np.pi).on(q[4]),
    cirq.rz(rads=13.750356038389338*np.pi).on(q[5]),
    cirq.rz(rads=-14.190869761261286*np.pi).on(q[6]),
    cirq.rz(rads=-10.440214577494247*np.pi).on(q[8]),
    cirq.rz(rads=10.5320224995019*np.pi).on(q[9]),
    cirq.rz(rads=2.0599079899133517*np.pi).on(q[10]),
    cirq.rz(rads=-2.067610652591436*np.pi).on(q[11]),
    cirq.rz(rads=8.780337110122234*np.pi).on(q[12]),
    cirq.rz(rads=-8.810594831167785*np.pi).on(q[13]),
    cirq.rz(rads=8.199075778124648*np.pi).on(q[14]),
    cirq.rz(rads=-8.07787223733251*np.pi).on(q[15]),
    cirq.rz(rads=-9.025823706766039*np.pi).on(q[16]),
    cirq.rz(rads=9.039336427865951*np.pi).on(q[17]),
    cirq.rz(rads=2.570829500938612*np.pi).on(q[19]),
    cirq.rz(rads=-2.5147874202965053*np.pi).on(q[20]),
    cirq.rz(rads=6.561341949396702*np.pi).on(q[21]),
    cirq.rz(rads=-6.48806953817391*np.pi).on(q[22]),
    cirq.rz(rads=10.239062711844038*np.pi).on(q[25]),
    cirq.rz(rads=-10.280506037093904*np.pi).on(q[26]),
    cirq.rz(rads=14.161779067835406*np.pi).on(q[27]),
    cirq.rz(rads=-14.136963411750049*np.pi).on(q[28]),
    cirq.rz(rads=13.413311792027148*np.pi).on(q[29]),
    cirq.rz(rads=-13.3218252900141*np.pi).on(q[30]),
    cirq.rz(rads=-30.29986888404229*np.pi).on(q[0]),
    cirq.rz(rads=30.302497289344824*np.pi).on(q[1]),
    cirq.rz(rads=-30.054485552499738*np.pi).on(q[4]),
    cirq.rz(rads=30.034988862096366*np.pi).on(q[5]),
    cirq.rz(rads=12.597359127188014*np.pi).on(q[7]),
    cirq.rz(rads=-12.538809626368511*np.pi).on(q[8]),
    cirq.rz(rads=-26.08870806725985*np.pi).on(q[9]),
    cirq.rz(rads=26.13388826789855*np.pi).on(q[10]),
    cirq.rz(rads=11.90395452297206*np.pi).on(q[13]),
    cirq.rz(rads=-11.950266520580142*np.pi).on(q[14]),
    cirq.rz(rads=-23.906047663366408*np.pi).on(q[15]),
    cirq.rz(rads=23.945335880513902*np.pi).on(q[16]),
    cirq.rz(rads=-12.64994198965531*np.pi).on(q[18]),
    cirq.rz(rads=12.670825603456805*np.pi).on(q[19]),
    cirq.rz(rads=5.221137480344522*np.pi).on(q[20]),
    cirq.rz(rads=-5.118027941327464*np.pi).on(q[21]),
    cirq.rz(rads=9.263573798570924*np.pi).on(q[22]),
    cirq.rz(rads=-9.55041239213535*np.pi).on(q[23]),
    cirq.rz(rads=8.765227495500554*np.pi).on(q[24]),
    cirq.rz(rads=-9.048084333301423*np.pi).on(q[25]),
    cirq.rz(rads=13.422682742974219*np.pi).on(q[26]),
    cirq.rz(rads=-13.34768684100077*np.pi).on(q[27]),
    cirq.rz(rads=7.346307418341885*np.pi).on(q[30]),
    cirq.rz(rads=-7.447034178043343*np.pi).on(q[31]),
    cirq.rz(rads=30.657856052586013*np.pi).on(q[0]),
    cirq.rz(rads=-30.65522764728348*np.pi).on(q[1]),
    cirq.rz(rads=30.087139057028068*np.pi).on(q[4]),
    cirq.rz(rads=-30.106635747431437*np.pi).on(q[5]),
    cirq.rz(rads=-12.410072351850753*np.pi).on(q[7]),
    cirq.rz(rads=12.468621852670255*np.pi).on(q[8]),
    cirq.rz(rads=26.656199817881895*np.pi).on(q[9]),
    cirq.rz(rads=-26.611019617243198*np.pi).on(q[10]),
    cirq.rz(rads=-12.422336273583753*np.pi).on(q[13]),
    cirq.rz(rads=12.376024275975672*np.pi).on(q[14]),
    cirq.rz(rads=24.032508849527318*np.pi).on(q[15]),
    cirq.rz(rads=-23.993220632379824*np.pi).on(q[16]),
    cirq.rz(rads=11.745064422304269*np.pi).on(q[18]),
    cirq.rz(rads=-11.724180808502775*np.pi).on(q[19]),
    cirq.rz(rads=-5.991658529535789*np.pi).on(q[20]),
    cirq.rz(rads=6.094768068552847*np.pi).on(q[21]),
    cirq.rz(rads=-9.293307215154037*np.pi).on(q[22]),
    cirq.rz(rads=9.006468621589612*np.pi).on(q[23]),
    cirq.rz(rads=-7.056913502275617*np.pi).on(q[24]),
    cirq.rz(rads=6.774056664474749*np.pi).on(q[25]),
    cirq.rz(rads=-13.45823318122632*np.pi).on(q[26]),
    cirq.rz(rads=13.53322908319977*np.pi).on(q[27]),
    cirq.rz(rads=-7.785583222537938*np.pi).on(q[30]),
    cirq.rz(rads=7.68485646283648*np.pi).on(q[31]),
    cirq.rz(rads=-26.934223110661993*np.pi).on(q[3]),
    cirq.rz(rads=26.964315883363945*np.pi).on(q[8]),
    cirq.rz(rads=-25.436295096608994*np.pi).on(q[4]),
    cirq.rz(rads=25.43183342244327*np.pi).on(q[9]),
    cirq.rz(rads=-21.351868774756507*np.pi).on(q[5]),
    cirq.rz(rads=21.24115658410936*np.pi).on(q[10]),
    cirq.rz(rads=-6.643807240413623*np.pi).on(q[6]),
    cirq.rz(rads=6.598519238690134*np.pi).on(q[11]),
    cirq.rz(rads=-21.096932498710586*np.pi).on(q[12]),
    cirq.rz(rads=21.084221422935954*np.pi).on(q[19]),
    cirq.rz(rads=-13.046692838556257*np.pi).on(q[13]),
    cirq.rz(rads=13.107006443219033*np.pi).on(q[20]),
    cirq.rz(rads=-20.486755870544844*np.pi).on(q[14]),
    cirq.rz(rads=20.51325689904871*np.pi).on(q[21]),
    cirq.rz(rads=-19.82814477311278*np.pi).on(q[15]),
    cirq.rz(rads=19.838469153514826*np.pi).on(q[22]),
    cirq.rz(rads=17.687652291347487*np.pi).on(q[16]),
    cirq.rz(rads=-17.230577602838448*np.pi).on(q[23]),
    cirq.rz(rads=27.146146626922736*np.pi).on(q[25]),
    cirq.rz(rads=-27.128766026200324*np.pi).on(q[29]),
    cirq.rz(rads=23.46087327851529*np.pi).on(q[26]),
    cirq.rz(rads=-23.443378195920936*np.pi).on(q[30]),
    cirq.rz(rads=17.157142369360066*np.pi).on(q[27]),
    cirq.rz(rads=-17.33618210685809*np.pi).on(q[31]),
    cirq.rz(rads=27.61944406229804*np.pi).on(q[3]),
    cirq.rz(rads=-27.589351289596088*np.pi).on(q[8]),
    cirq.rz(rads=25.746615138480117*np.pi).on(q[4]),
    cirq.rz(rads=-25.75107681264584*np.pi).on(q[9]),
    cirq.rz(rads=22.13450605727*np.pi).on(q[5]),
    cirq.rz(rads=-22.245218247917148*np.pi).on(q[10]),
    cirq.rz(rads=7.440729485113184*np.pi).on(q[6]),
    cirq.rz(rads=-7.486017486836674*np.pi).on(q[11]),
    cirq.rz(rads=20.425705877923868*np.pi).on(q[12]),
    cirq.rz(rads=-20.4384169536985*np.pi).on(q[19]),
    cirq.rz(rads=12.71050872566315*np.pi).on(q[13]),
    cirq.rz(rads=-12.650195121000372*np.pi).on(q[20]),
    cirq.rz(rads=19.562531408750814*np.pi).on(q[14]),
    cirq.rz(rads=-19.53603038024695*np.pi).on(q[21]),
    cirq.rz(rads=18.69447149644039*np.pi).on(q[15]),
    cirq.rz(rads=-18.684147116038343*np.pi).on(q[22]),
    cirq.rz(rads=-18.21701654221441*np.pi).on(q[16]),
    cirq.rz(rads=18.674091230723448*np.pi).on(q[23]),
    cirq.rz(rads=-25.757287261948953*np.pi).on(q[25]),
    cirq.rz(rads=25.774667862671368*np.pi).on(q[29]),
    cirq.rz(rads=-23.52062771406781*np.pi).on(q[26]),
    cirq.rz(rads=23.538122796662165*np.pi).on(q[30]),
    cirq.rz(rads=-16.556589282442097*np.pi).on(q[27]),
    cirq.rz(rads=16.377549544944078*np.pi).on(q[31]),
    cirq.rz(rads=-6.926584587366442*np.pi).on(q[0]),
    cirq.rz(rads=6.929081406888282*np.pi).on(q[4]),
    cirq.rz(rads=-6.864078115073335*np.pi).on(q[1]),
    cirq.rz(rads=6.859398026235534*np.pi).on(q[5]),
    cirq.rz(rads=7.418125678549155*np.pi).on(q[2]),
    cirq.rz(rads=-7.424456163642465*np.pi).on(q[6]),
    cirq.rz(rads=39.34359792383697*np.pi).on(q[7]),
    cirq.rz(rads=-39.42696290791402*np.pi).on(q[13]),
    cirq.rz(rads=38.52825306393881*np.pi).on(q[8]),
    cirq.rz(rads=-38.452584830758425*np.pi).on(q[14]),
    cirq.rz(rads=12.958562501914345*np.pi).on(q[9]),
    cirq.rz(rads=-13.049596611893888*np.pi).on(q[15]),
    cirq.rz(rads=15.428639843256777*np.pi).on(q[10]),
    cirq.rz(rads=-15.42731467518594*np.pi).on(q[16]),
    cirq.rz(rads=28.031937308298577*np.pi).on(q[11]),
    cirq.rz(rads=-28.01590894186218*np.pi).on(q[17]),
    cirq.rz(rads=6.967481743922609*np.pi).on(q[18]),
    cirq.rz(rads=-7.259908389782927*np.pi).on(q[24]),
    cirq.rz(rads=31.210231816728815*np.pi).on(q[19]),
    cirq.rz(rads=-31.213639326033913*np.pi).on(q[25]),
    cirq.rz(rads=19.946499004273523*np.pi).on(q[20]),
    cirq.rz(rads=-19.833707101903418*np.pi).on(q[26]),
    cirq.rz(rads=30.137045801919207*np.pi).on(q[21]),
    cirq.rz(rads=-30.211658619314296*np.pi).on(q[27]),
    cirq.rz(rads=21.231104618355*np.pi).on(q[22]),
    cirq.rz(rads=-21.186606163793456*np.pi).on(q[28]),
    cirq.rz(rads=6.997874433792849*np.pi).on(q[0]),
    cirq.rz(rads=-6.995377614271008*np.pi).on(q[4]),
    cirq.rz(rads=6.418995232832726*np.pi).on(q[1]),
    cirq.rz(rads=-6.423675321670527*np.pi).on(q[5]),
    cirq.rz(rads=-7.8378074360795305*np.pi).on(q[2]),
    cirq.rz(rads=7.831476950986221*np.pi).on(q[6]),
    cirq.rz(rads=-39.11725021952874*np.pi).on(q[7]),
    cirq.rz(rads=39.03388523545169*np.pi).on(q[13]),
    cirq.rz(rads=-37.65488974563269*np.pi).on(q[8]),
    cirq.rz(rads=37.730557978813074*np.pi).on(q[14]),
    cirq.rz(rads=-14.665863640797525*np.pi).on(q[9]),
    cirq.rz(rads=14.574829530817984*np.pi).on(q[15]),
    cirq.rz(rads=-16.519871460773594*np.pi).on(q[10]),
    cirq.rz(rads=16.52119662884443*np.pi).on(q[16]),
    cirq.rz(rads=-28.386693052252454*np.pi).on(q[11]),
    cirq.rz(rads=28.402721418688852*np.pi).on(q[17]),
    cirq.rz(rads=-7.731815524796781*np.pi).on(q[18]),
    cirq.rz(rads=7.439388878936463*np.pi).on(q[24]),
    cirq.rz(rads=-31.64256101076613*np.pi).on(q[19]),
    cirq.rz(rads=31.63915350146103*np.pi).on(q[25]),
    cirq.rz(rads=-19.050421382024783*np.pi).on(q[20]),
    cirq.rz(rads=19.16321328439489*np.pi).on(q[26]),
    cirq.rz(rads=-29.465079763839764*np.pi).on(q[21]),
    cirq.rz(rads=29.390466946444676*np.pi).on(q[27]),
    cirq.rz(rads=-21.367003503847553*np.pi).on(q[22]),
    cirq.rz(rads=21.411501958409097*np.pi).on(q[28]),
    cirq.rz(rads=-34.93587030317863*np.pi).on(q[1]),
    cirq.rz(rads=34.899691840749924*np.pi).on(q[2]),
    cirq.rz(rads=13.66913916974463*np.pi).on(q[3]),
    cirq.rz(rads=-13.714527436529053*np.pi).on(q[4]),
    cirq.rz(rads=-19.300997501357458*np.pi).on(q[5]),
    cirq.rz(rads=18.86048377848551*np.pi).on(q[6]),
    cirq.rz(rads=14.928737337703097*np.pi).on(q[8]),
    cirq.rz(rads=-14.836929415695444*np.pi).on(q[9]),
    cirq.rz(rads=-2.1976740888018944*np.pi).on(q[10]),
    cirq.rz(rads=2.1899714261238103*np.pi).on(q[11]),
    cirq.rz(rads=-12.39075098081413*np.pi).on(q[12]),
    cirq.rz(rads=12.360493259768578*np.pi).on(q[13]),
    cirq.rz(rads=-12.10125113388289*np.pi).on(q[14]),
    cirq.rz(rads=12.22245467467503*np.pi).on(q[15]),
    cirq.rz(rads=10.936894386213037*np.pi).on(q[16]),
    cirq.rz(rads=-10.923381665113125*np.pi).on(q[17]),
    cirq.rz(rads=-2.8894238777188748*np.pi).on(q[19]),
    cirq.rz(rads=2.945465958360982*np.pi).on(q[20]),
    cirq.rz(rads=-10.099134633961603*np.pi).on(q[21]),
    cirq.rz(rads=10.172407045184396*np.pi).on(q[22]),
    cirq.rz(rads=-15.060126243969762*np.pi).on(q[25]),
    cirq.rz(rads=15.018682918719897*np.pi).on(q[26]),
    cirq.rz(rads=-18.34652096929912*np.pi).on(q[27]),
    cirq.rz(rads=18.371336625384476*np.pi).on(q[28]),
    cirq.rz(rads=-19.622795435376638*np.pi).on(q[29]),
    cirq.rz(rads=19.714281937389686*np.pi).on(q[30]),
    cirq.rz(rads=34.236310550447435*np.pi).on(q[1]),
    cirq.rz(rads=-34.27248901287614*np.pi).on(q[2]),
    cirq.rz(rads=-13.19807055510896*np.pi).on(q[3]),
    cirq.rz(rads=13.152682288324536*np.pi).on(q[4]),
    cirq.rz(rads=19.96635603838931*np.pi).on(q[5]),
    cirq.rz(rads=-20.40686976126126*np.pi).on(q[6]),
    cirq.rz(rads=-15.32421457749417*np.pi).on(q[8]),
    cirq.rz(rads=15.416022499501823*np.pi).on(q[9]),
    cirq.rz(rads=2.7999079899133363*np.pi).on(q[10]),
    cirq.rz(rads=-2.80761065259142*np.pi).on(q[11]),
    cirq.rz(rads=12.628337110122207*np.pi).on(q[12]),
    cirq.rz(rads=-12.658594831167758*np.pi).on(q[13]),
    cirq.rz(rads=11.899075778124569*np.pi).on(q[14]),
    cirq.rz(rads=-11.777872237332431*np.pi).on(q[15]),
    cirq.rz(rads=-12.725823706766091*np.pi).on(q[16]),
    cirq.rz(rads=12.739336427866004*np.pi).on(q[17]),
    cirq.rz(rads=3.458829500938646*np.pi).on(q[19]),
    cirq.rz(rads=-3.4027874202965385*np.pi).on(q[20]),
    cirq.rz(rads=9.817341949396608*np.pi).on(q[21]),
    cirq.rz(rads=-9.744069538173814*np.pi).on(q[22]),
    cirq.rz(rads=15.12306271184396*np.pi).on(q[25]),
    cirq.rz(rads=-15.164506037093826*np.pi).on(q[26]),
    cirq.rz(rads=20.160375777985994*np.pi).on(q[27]),
    cirq.rz(rads=-20.13556012190064*np.pi).on(q[28]),
    cirq.rz(rads=19.481311792027203*np.pi).on(q[29]),
    cirq.rz(rads=-19.389825290014155*np.pi).on(q[30]),
    cirq.rz(rads=-42.731868884042235*np.pi).on(q[0]),
    cirq.rz(rads=42.73449728934477*np.pi).on(q[1]),
    cirq.rz(rads=-42.48648555249982*np.pi).on(q[4]),
    cirq.rz(rads=42.46698886209646*np.pi).on(q[5]),
    cirq.rz(rads=17.629359127188117*np.pi).on(q[7]),
    cirq.rz(rads=-17.570809626368614*np.pi).on(q[8]),
    cirq.rz(rads=-36.89270806725978*np.pi).on(q[9]),
    cirq.rz(rads=36.93788826789848*np.pi).on(q[10]),
    cirq.rz(rads=16.787954522971983*np.pi).on(q[13]),
    cirq.rz(rads=-16.834266520580062*np.pi).on(q[14]),
    cirq.rz(rads=-33.970047663366486*np.pi).on(q[15]),
    cirq.rz(rads=34.00933588051398*np.pi).on(q[16]),
    cirq.rz(rads=-17.533941989655233*np.pi).on(q[18]),
    cirq.rz(rads=17.554825603456727*np.pi).on(q[19]),
    cirq.rz(rads=7.441137480344476*np.pi).on(q[20]),
    cirq.rz(rads=-7.338027941327417*np.pi).on(q[21]),
    cirq.rz(rads=12.963573798570843*np.pi).on(q[22]),
    cirq.rz(rads=-13.250412392135269*np.pi).on(q[23]),
    cirq.rz(rads=12.021227495500458*np.pi).on(q[24]),
    cirq.rz(rads=-12.30408433330133*np.pi).on(q[25]),
    cirq.rz(rads=18.97727945312479*np.pi).on(q[26]),
    cirq.rz(rads=-18.902283551151342*np.pi).on(q[27]),
    cirq.rz(rads=10.306307418341955*np.pi).on(q[30]),
    cirq.rz(rads=-10.407034178043412*np.pi).on(q[31]),
    cirq.rz(rads=43.08985605258596*np.pi).on(q[0]),
    cirq.rz(rads=-43.08722764728342*np.pi).on(q[1]),
    cirq.rz(rads=42.51913905702814*np.pi).on(q[4]),
    cirq.rz(rads=-42.53863574743151*np.pi).on(q[5]),
    cirq.rz(rads=-17.442072351850854*np.pi).on(q[7]),
    cirq.rz(rads=17.500621852670356*np.pi).on(q[8]),
    cirq.rz(rads=37.46019981788182*np.pi).on(q[9]),
    cirq.rz(rads=-37.415019617243125*np.pi).on(q[10]),
    cirq.rz(rads=-17.306336273583675*np.pi).on(q[13]),
    cirq.rz(rads=17.260024275975592*np.pi).on(q[14]),
    cirq.rz(rads=34.09650884952739*np.pi).on(q[15]),
    cirq.rz(rads=-34.057220632379895*np.pi).on(q[16]),
    cirq.rz(rads=16.629064422304193*np.pi).on(q[18]),
    cirq.rz(rads=-16.6081808085027*np.pi).on(q[19]),
    cirq.rz(rads=-8.211658529535743*np.pi).on(q[20]),
    cirq.rz(rads=8.3147680685528*np.pi).on(q[21]),
    cirq.rz(rads=-12.993307215153958*np.pi).on(q[22]),
    cirq.rz(rads=12.706468621589535*np.pi).on(q[23]),
    cirq.rz(rads=-10.31291350227552*np.pi).on(q[24]),
    cirq.rz(rads=10.030056664474653*np.pi).on(q[25]),
    cirq.rz(rads=-19.012829891376892*np.pi).on(q[26]),
    cirq.rz(rads=19.08782579335034*np.pi).on(q[27]),
    cirq.rz(rads=-10.745583222538006*np.pi).on(q[30]),
    cirq.rz(rads=10.644856462836547*np.pi).on(q[31]),
    ],
    strategy=cirq.InsertStrategy.EARLIEST)

# print a representation of the circuit

print(circuit)

(1, 5): ───Rz(-1.44π)────Rz(1.79π)─────Rz(1.51π)─────Rz(-1.44π)────Rz(-1.87π)────Rz(-1.77π)────Rz(-0.707π)───Rz(0.778π)────Rz(1.7π)──────Rz(-1.34π)────Rz(1.07π)─────Rz(-1.0π)─────Rz(1.27π)─────Rz(-0.91π)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

(1, 6): ───Rz(1.92π)─────Rz(1.38π)─────Rz(1.44π)─────Rz(-1.79π)────Rz(1.58π)─────Rz(1.98π)─────Rz(-1.03π)────Rz(0.332π)────Rz(1.87π)─────Rz(1.78π)─────Rz(-0.644π)───Rz(0.199π)────Rz(0.016π)────Rz(-0.716π)───Rz(-1.7π)─────Rz(1.34π)─────Rz(1.14π)─────Rz(-1.58π)────Rz(1.06π)─────Rz(-1.76π)────Rz(-1.27π)────Rz(0.913π)────────────────────────────────────────────────────────────────────────────────────────

(1, 7): ───Rz(-1.96π)────Rz(-1.42π)────Rz(-1.61π)────Rz(1.19π)─────Rz(0.996π)────Rz(-0.368π)───Rz(0.902π)────Rz(-1.32π)────Rz(-0.052π)───Rz(0.68π)─────Rz(-0.582π)───Rz(0.162π)────Rz(-1.1π)──

## Run the circuit

Run the amplitude calculation. It should be a few minutes:

In [6]:
options = {'t': 16, 'v': 3}
simulator = qsimcirq.QSimSimulator(options)

wall_start = time.monotonic()
process_start = time.process_time()

results = simulator.compute_amplitudes(circuit,
                                       bitstrings=[0b11011001001100111101110001000001,
                                                   0b11011011000111010001001010001011])

wall_end = time.monotonic()
process_end = time.process_time()

print(f"Result: {results}")

total_process_time = process_end - process_start
total_wall_time = wall_end - wall_start

print(f"Wall Time: {total_wall_time}   -- Process Time: {total_process_time} ")

Result: [0j, 0j]
Wall Time: 17.303243874999993   -- Process Time: 155.977301248 
